# Check if the packet loss data is abnormal

###This notebook finds out all the links which have at least five packet loss measurements in the past one hour and the average value of the packet loss measurements is greater than 2%. It is run by a cron job every hour, and it will write the detailed information of every alarm into Elastic Search with the _index: alarms-year.month and _type: packetloss.

### import all the packages needed for this task

In [1]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions, helpers
import sys
import datetime

### establish the Elastic Search connection

In [2]:
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

### define functions to write an alarm record into ES with detailed info

In [3]:
ipSite={}  # for mapping IP to Site name
toAlertOn=[]

def generate_doc(src_site_ip, dest_site_ip, measurements, avgpl):
   if src_site_ip not in ipSite:
        print('serious source mapping issue')
        return
   if dest_site_ip not in ipSite:
        print('serious destination mapping issue')
        return

   doc = {
        '_index':get_index_name(),
        '_type' : 'packetloss',
        'src' : src_site_ip,
        'dest' : dest_site_ip,
        'srcSite' : ipSite[src_site_ip],
        'destSite' : ipSite[dest_site_ip],
        'alarmTime' : int(datetime.datetime.utcnow().strftime("%s"))*1000,
        'measurements' : measurements,
        'packetLossAvg' : avgpl
   }
   return doc

def get_index_name():
    date = datetime.datetime.utcnow().strftime("%Y.%m")   # date format is yyyy.mm
    index_name = 'alarms-'+date
    return index_name



### list all network\_weather* indices

In [4]:
indices = es.cat.indices(index="network_weather_2-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
#print(indices)

### find indices to be used

In [5]:

cday  = datetime.datetime.utcnow()
pday  = cday - datetime.timedelta(days=1)
ind1 = 'network_weather_2-%d.%d.%d' % (cday.year, cday.month, cday.day)
ind2 = 'network_weather_2-%d.%d.%d' % (pday.year, pday.month, pday.day)

print ('checking for indices:', ind1, ind2)

ind=[]
if ind1 in indices :
   ind.append(ind1)
if ind2 in indices and cday.hour<3:
   ind.append(ind2)

if len(ind)==0:
   print ('no current indices found. Aborting.')
   sys.exit(1)
else:
   print('will use indices:', ind)

checking for indices: network_weather_2-2016.8.15 network_weather_2-2016.8.14
will use indices: ['network_weather_2-2016.8.15']


### get aggregated data for the past one hour
This query is composed of 3 parts: a) filter - takes only packet loss data, and production servers in last 1h. b) aggregation -  finds average packet loss per source and destination c) finds IP to site name mapping (both source and destination) 

In [6]:
query={
   "size": 0,
   "query": {
    "bool": {
      "must": [
        {"term": { "_type" : "packet_loss_rate"}},
        {"term": { "srcProduction" : True }},
        {"term": { "destProduction" : True }}
      ],
      "filter" : {
        "range" : {
          "timestamp" : {
            "gt": "now-1h"
          }
        }
      }
    }
   },
    "aggs" : {
      "src" : {
        "terms" : { "field" : "src", "size": 1000 },
        "aggs" : {
          "dest" : {
            "terms" : {"field" : "dest", "size": 1000},
            "aggs" : {
              "avgpl" : {
                "avg" :{
                  "field" : "packet_loss"
              }
            }
          }
        }
      }
    },
    "srcSites" : {
      "terms" : { "field" : "src", "size": 1000 },
        "aggs" : {
          "srcsitename" : {
            "terms" : { "field" : "srcSite" }
        }
      }
    },
    "destSites" : {
      "terms" : { "field" : "dest", "size": 1000 },
        "aggs" : {
          "destsitename" : {
            "terms" : { "field" : "destSite" }
        }
      }
    }
  }
}

### execute query

In [7]:
res = es.search(index=ind, body=query, request_timeout=120)
#print(res)

### proces IP to site name mapping data

In [8]:
srcsites=res['aggregations']['srcSites']['buckets']
#print(srcsites)
for sS in srcsites:
   #print(sS)
   siteName=sS['srcsitename']['buckets']
   if len(siteName)==0:
      siteName='UnknownSite'
   else:
      siteName=siteName[0]['key']
   ipSite[sS['key']]=siteName

destsites=res['aggregations']['destSites']['buckets']
#print(destsites)
for dS in destsites:
   #print(dS)
   siteName=dS['destsitename']['buckets']
   if len(siteName)==0:
      siteName='UnknownSite'
   else:
      siteName=siteName[0]['key']
   ipSite[dS['key']]=siteName

print(ipSite)


{'134.158.73.243': 'GRIF', '134.79.118.72': 'WT2', '193.109.172.188': 'pic', '134.158.103.10': 'IN2P3-LAPP', '192.135.14.32': 'INFN-MILANO-ATLASC', '194.36.11.38': 'UKI-LT2-QMUL', '193.170.243.215': 'Hephy-Vienna', '64.57.30.143': 'UnknownSite', '134.158.123.183': 'IN2P3-LPC', '147.231.25.192': 'praguelcg2', '144.92.180.75': 'GLOW', '134.219.225.13': 'UKI-LT2-RHUL', '128.227.221.44': 'UFlorida-HPC', '159.93.229.151': 'JINR-T1', '131.154.254.12': 'INFN-T1', '145.100.17.8': 'SARA-MATRIX', '90.147.66.50': 'UnknownSite', '144.16.111.26': 'INDIACMS-TIFR', '192.84.86.122': 'UnknownSite', '165.91.55.4': 'UnknownSite', '149.165.225.223': 'MWT2', '147.156.116.40': 'IFIC-LCG2', '141.34.200.28': 'DESY-ZN', '192.12.15.111': 'BNL-ATLAS', '192.170.227.163': 'UnknownSite', '193.48.83.165': 'IN2P3-LPSC', '193.146.75.138': 'IFCA-LCG2', '143.215.129.69': 'UnknownSite', '192.101.107.152': 'UnknownSite', '129.107.255.26': 'UnknownSite', '147.213.204.112': 'IEPSAS-Kosice', '131.169.98.30': 'DESY-HH', '147.

### process packet loss averages

In [10]:
src=res['aggregations']['src']['buckets']
#print(src)

for s in src:
   #print(s)
   source=s['key']
   for d in s['dest']['buckets']:
      destination=d['key']
      avgpl=d['avgpl']['value']
      docs=d['doc_count']
#      print(source, destination, docs, avgpl)
      if avgpl > 0.02 and docs > 4:
         toAlertOn.append(generate_doc(source, destination, docs, avgpl))

for alert in toAlertOn:
   print(alert)

{'_type': 'packetloss', 'src': '134.158.73.243', 'srcSite': 'GRIF', 'alarmTime': 1471291658000, 'destSite': 'UnknownSite', 'dest': '161.116.81.235', '_index': 'alarms-2016.08', 'measurements': 22, 'packetLossAvg': 1.0}
{'_type': 'packetloss', 'src': '134.158.73.243', 'srcSite': 'GRIF', 'alarmTime': 1471291658000, 'destSite': 'NCG-INGRID-PT', 'dest': '193.136.75.146', '_index': 'alarms-2016.08', 'measurements': 8, 'packetLossAvg': 0.7749999999999999}
{'_type': 'packetloss', 'src': '134.158.73.243', 'srcSite': 'GRIF', 'alarmTime': 1471291658000, 'destSite': 'ITEP', 'dest': '194.85.69.75', '_index': 'alarms-2016.08', 'measurements': 8, 'packetLossAvg': 0.08237570032545172}
{'_type': 'packetloss', 'src': '134.158.73.243', 'srcSite': 'GRIF', 'alarmTime': 1471291658000, 'destSite': 'SPRACE', 'dest': '200.136.80.20', '_index': 'alarms-2016.08', 'measurements': 8, 'packetLossAvg': 0.02975}
{'_type': 'packetloss', 'src': '194.190.165.192', 'srcSite': 'RU-Protvino-IHEP', 'alarmTime': 14712916580

### write alarms to Elasticsearch

In [11]:
try:
   res = helpers.bulk(es, toAlertOn, raise_on_exception=True,request_timeout=60)
   print("inserted:",res[0], '\tErrors:',res[1])
except es_exceptions.ConnectionError as e:
   print('ConnectionError ', e)
except es_exceptions.TransportError as e:
   print('TransportError ', e)
except helpers.BulkIndexError as e:
   print(e[0])
   for i in e[1]:
      print(i)
except:
   print('Something seriously wrong happened.')

inserted: 205 	Errors: []
